In [12]:
import sys 
sys.path.append('./../src')

from lib.stonk_jar import StonkJar

In [28]:
ticker = 'GME'
jar = StonkJar(ticker)
historical_pickle_name = "{0}.technical.D.historical.df.pkl".format(ticker)

data = jar.read_pickle_dataframe(historical_pickle_name)

data.index = data['ts']
data.sort_index()
data.drop_duplicates()

,ts,o,l,h,c,v,e_a,e_e,r_ss,r_s,r_h,r_b,r_sb
ts,,,,,,,,,,,,,
2020-12-01 00:00:00,2020-12-01 00:00:00,120.01,121.01,122.720,123.4693,128166803,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2020-12-02 00:00:00,2020-12-02 00:00:00,120.89,122.02,123.080,123.3700,89004195,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2020-12-03 00:00:00,2020-12-03 00:00:00,122.21,123.52,122.940,123.7800,78967630,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2020-12-04 00:00:00,2020-12-04 00:00:00,121.52,122.60,122.250,122.8608,78260421,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2020-12-07 00:00:00,2020-12-07 00:00:00,122.25,122.31,123.750,124.5700,86711990,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04 00:00:00,2021-03-04 00:00:00,118.62,121.75,120.130,123.6000,177275300,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-05 14:30:00,2021-03-05 14:30:00,117.57,120.98,118.900,121.6700,71986254,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-05 14:30:00,2021-03-05 14:30:00,117.57,120.98,118.899,121.6700,71996083,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0


In [29]:
data.describe()

,o,l,h,c,v,e_a,e_e,r_ss,r_s,r_h,r_b,r_sb
count,1794.000000,1794.000000,1794.000000,1794.000000,1.794000e+03,1.794000e+03,1794.000000,1794.0,1794.0,1794.0,1794.0,1794.0
mean,128.691088,130.574125,130.510833,132.123825,1.072568e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
std,5.532129,5.827971,5.596367,5.705862,3.031395e+07,2.221065e-16,0.000000,0.0,0.0,0.0,0.0,0.0
min,117.570000,120.500000,118.899000,121.670000,5.493006e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
25%,123.449000,124.940000,126.600000,127.930000,8.693979e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
50%,128.500000,129.200000,130.890000,131.685000,9.839056e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
75%,133.590000,135.580000,134.870000,136.310000,1.210473e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
max,141.370000,143.600000,143.160000,145.090000,1.925415e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0


In [16]:
def sma(vals):
    return sum(vals) / len(vals)

In [17]:
def fastk(vals):
    t_close = vals[0]
    t_high = max(vals)
    t_low = min(vals)
    return (t_close - t_low) / (t_high - t_low)

In [18]:
def fastd(vals):
    return smooth(fastk, sma, vals) / len(vals)

In [19]:
def slowk(vals):
    return smooth(fastk, sma, vals) / len(vals)

In [20]:
def slowd(vals):
    return smooth(slowk, sma, vals) / len(vals)

In [21]:
def smooth(fn1, fn2, vals):
    d = fn2(vals)
    return fn1(list(map(lambda v: v / d, vals)))

In [23]:
def stochastic(vals, fastk_days = 7, slowk_days = 3, slowd_days = 3):
    if len(vals) < max(fastk_days, max(slowk_days, slowd_days)):
        print("not enough vals for requested day config")
        return None
    print(vals[0:fastk_days])
    fastk_val = fastk(vals[0:fastk_days])
    slowk_val = slowk(vals[0:slowk_days])
    slowd_val = slowd(vals[0:slowd_days])
    return [fastk_val, slowk_val, slowd_val]

In [24]:
stochastic(list(data.tail(90)['c']))

[136.01, 132.22, 129.995, 130.71, 129.72, 126.71, 125.56]


[1.0, 0.3333333333333333, 0.1111111111111111]

In [30]:
data.tail(90).tail(7)

,ts,o,l,h,c,v,e_a,e_e,r_ss,r_s,r_h,r_b,r_sb
ts,,,,,,,,,,,,,
2021-03-04 00:00:00,2021-03-04 00:00:00,118.62,121.75,120.13,123.60,177275300,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-05 14:30:00,2021-03-05 14:30:00,117.57,120.98,118.95,121.67,72024122,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-01 00:00:00,2021-03-01 00:00:00,122.79,123.75,127.79,127.93,116307892,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-02 00:00:00,2021-03-02 00:00:00,125.01,128.41,125.12,128.72,102260945,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-03 00:00:00,2021-03-03 00:00:00,121.84,124.81,122.06,125.71,112966340,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-04 00:00:00,2021-03-04 00:00:00,118.62,121.75,120.13,123.60,177275300,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
2021-03-05 14:30:00,2021-03-05 14:30:00,117.57,120.98,118.93,121.67,72056846,-0.53,-0.857116,0.0,0.0,0.0,0.0,0.0
